In [1]:
from sqlalchemy import create_engine
import pymysql
import pandas as pd


In [2]:
# con = pymysql.connect(host='localhost', user='root', password='1234',
#                       db='MySQl', charset='utf8', # 한글처리 (charset = 'utf8')
#                       autocommit=True, # 결과 DB 반영 (Insert or update)
#                       cursorclass=pymysql.cursors.DictCursor # DB조회시 컬럼명을 동시에 보여줌
#                      )
# con.cursor()

In [3]:
engine = create_engine("mysql+pymysql://root:"
            +"1234"
            +"@localhost:3306/MySQL?charset=utf8"
            , encoding='utf-8')

In [4]:
conn = engine.connect()

In [5]:
db = pd.read_csv('../data/000250_삼천당제약.csv')

In [6]:
db.to_sql(name='stock_info', con=engine, if_exists='append', index=False)

181201

In [7]:
conn.close()

In [8]:
host_name = "localhost"
username = 'root'
password = '1234'
database_name = 'stock_info'

In [9]:
db = pymysql.connect(
    host= host_name
    , port=3306
    , user=username
    , password=password
    , db=database_name
    , charset='utf8'
)

In [10]:
cur = db.cursor()

In [11]:
indata = pd.read_sql_query('select * from stock_info', engine)
indata

MemoryError: Unable to allocate 138. MiB for an array with shape (20, 906005) and data type int64

In [12]:
db.commit()

In [13]:
db.close()

In [14]:
cur.fetchall()

ProgrammingError: execute() first

In [ ]:
# cur = con.cursor()
# # sql = "SELECT * FROM stock_info.stock" # customers 테이블 전체를 불러옴
# sql = "SELECT * FROM stock_info.stock.columns" # customers 테이블 전체를 불러옴
# cur.execute(sql)
# rows = cur.fetchone()
# con.close() # DB 연결 종료
# print(rows)

In [ ]:
# test = pd.read_csv('../data/000250_삼천당제약.csv')
# # test
# test.to_sql(name='trad',con=con, if_exists='append', index=False)

c:\Users\TJ\anaconda3\envs\trad\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


DatabaseError: Execution failed on sql 'SELECT name FROM sqlite_master WHERE type='table' AND name=?;': not all arguments converted during string formatting